In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense

df = pd.read_csv('/content/drive/MyDrive/zomato.csv')
df.drop(['url', 'phone', 'dish_liked', 'menu_item'], axis=1, inplace=True)
df.drop_duplicates(inplace=True)

df.dropna(inplace=True)

def convert_rate_to_numeric(rate):
    if isinstance(rate, str) and '/' in rate:
        return float(rate.split('/')[0])
    else:
        return np.nan

df['rate'] = df['rate'].apply(convert_rate_to_numeric)

df.dropna(subset=['rate'], inplace=True)

df['approx_cost(for two people)'] = df['approx_cost(for two people)'].str.replace(',', '')

df['approx_cost(for two people)'] = pd.to_numeric(df['approx_cost(for two people)'], errors='coerce')

df.dropna(subset=['approx_cost(for two people)'], inplace=True)

scaler = MinMaxScaler()
df[['rate', 'approx_cost(for two people)', 'votes']] = scaler.fit_transform(df[['rate', 'approx_cost(for two people)', 'votes']])

X = df[['rate', 'approx_cost(for two people)', 'votes']]
y = df['rate']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1)

mse = model.evaluate(X_test, y_test)
print("Mean Squared Error:", mse)
predicted_ratings = model.predict(X)

df['predicted_rating'] = predicted_ratings

recommended_restaurants = df.sort_values(by='predicted_rating', ascending=False)

print("Top Recommended Restaurants:")
print(recommended_restaurants[['name', 'location', 'rate', 'predicted_rating']].head(10))


Epoch 1/10
464/464 [==============================] - 2s 2ms/step - loss: 0.0087 - val_loss: 4.0945e-06
Epoch 2/10
464/464 [==============================] - 1s 2ms/step - loss: 1.9583e-06 - val_loss: 5.0710e-07
Epoch 3/10
464/464 [==============================] - 1s 2ms/step - loss: 3.5019e-07 - val_loss: 2.0495e-07
Epoch 4/10
464/464 [==============================] - 1s 2ms/step - loss: 2.0506e-07 - val_loss: 2.0010e-07
Epoch 5/10
464/464 [==============================] - 1s 2ms/step - loss: 1.6543e-07 - val_loss: 3.1956e-07
Epoch 6/10
464/464 [==============================] - 1s 2ms/step - loss: 1.3675e-07 - val_loss: 8.7244e-08
Epoch 7/10
464/464 [==============================] - 1s 2ms/step - loss: 1.5993e-07 - val_loss: 1.1102e-06
Epoch 8/10
464/464 [==============================] - 1s 2ms/step - loss: 1.1146e-07 - val_loss: 1.9202e-07
Epoch 9/10
464/464 [==============================] - 1s 2ms/step - loss: 1.7631e-07 - val_loss: 1.0707e-07
Epoch 10/10
258/258 [===========